# 1. Utilties

## 1.1 All files are available at this link
### https://drive.google.com/drive/folders/1k1nDWr2p4p63X2LegnFjBwqRhmjBl-Uh?usp=sharing

## 1.2 System Specs

In [ ]:
# Making sure how much of the available RAM is the notebook using when run on Google Colab
import sys
if 'google.colab' in sys.modules:
  from psutil import virtual_memory
  ram_gb = virtual_memory().total / 1e9
  print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

  if ram_gb < 20:
    print('Not using a high-RAM runtime')
  else:
    print('You are using a high-RAM runtime!')

Your runtime has 13.6 gigabytes of available RAM

Not using a high-RAM runtime


## 1.3 Imports and Downloads

In [ ]:
!pip install pandas spacy nltk autocorrect

import re
import csv
import numpy as np
import pandas as pd 
import spacy

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from nltk.stem import PorterStemmer,WordNetLemmatizer
from wordcloud import WordCloud
import unicodedata

from gensim.parsing.preprocessing import STOPWORDS as SW
from wordcloud import WordCloud, STOPWORDS
stopwords = set(STOPWORDS)
from bs4 import BeautifulSoup
import string 
from os import listdir
from autocorrect import Speller 

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')

     |████████████████████████████████| 622 kB 11.9 MB/s 
  Created wheel for autocorrect: filename=autocorrect-2.6.1-py3-none-any.whl size=622382 sha256=5d462b59469770d4a1fc5f0157f2eb460120591bdf959a9ad782f4129b0cd79c
  Stored in directory: /root/.cache/pip/wheels/54/d4/37/8244101ad50b0f7d9bffd93ce58ed7991ee1753b290923934b
Successfully built autocorrect
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.


True

#Load and Prepare Dataset

## 2.1 Reading & Extracting Transcript

In [ ]:
dataset= pd.read_csv('Transcripts.csv')

In [ ]:
dataset.head(7)

,title,transcript
0,astronomy,astronomy literally meaning the science that s...
1,white dwarf,a white dwarf also called a degenerate dwarf i...
2,chimp,chimpanzees are great apes found across centra...
3,crypto,what are the top four crypto exchanges india w...
4,crypto_tax,india recently introduced crypto tax in the bu...
5,crypto_means,crypto is just one big ponzi and can change my...
6,digital_token,token from your next the word pawnshop in lieu...


In [ ]:
dataset.shape

(25, 2)

In [ ]:
dataset.info

<bound method DataFrame.info of                 title                                         transcript
0           astronomy  astronomy literally meaning the science that s...
1         white dwarf  a white dwarf also called a degenerate dwarf i...
2               chimp  chimpanzees are great apes found across centra...
3              crypto  what are the top four crypto exchanges india w...
4          crypto_tax  india recently introduced crypto tax in the bu...
5        crypto_means  crypto is just one big ponzi and can change my...
6       digital_token  token from your next the word pawnshop in lieu...
7      cryptocurrency  a run up in cryptocurrency prices last decembe...
8                dodo  the dodo is an extinct flightless bird that wa...
9             dolphin  dolphins are small toothed cetaceans easily re...
10           elephant  elephants are the largest land mammals on eart...
11  fd_mutual_finance  bank fixed deposits and government small savin...
12            finan

In [ ]:
dataset.dtypes

title         object
transcript    object
dtype: object

In [ ]:
df = pd.DataFrame(columns=['transcript'])
df['transcript'] = dataset['transcript'].astype(str)
df[:5]

,transcript
0,astronomy literally meaning the science that s...
1,a white dwarf also called a degenerate dwarf i...
2,chimpanzees are great apes found across centra...
3,what are the top four crypto exchanges india w...
4,india recently introduced crypto tax in the bu...


In [ ]:
all_transcripts = dataset.transcript
all_transcripts

0     astronomy literally meaning the science that s...
1     a white dwarf also called a degenerate dwarf i...
2     chimpanzees are great apes found across centra...
3     what are the top four crypto exchanges india w...
4     india recently introduced crypto tax in the bu...
5     crypto is just one big ponzi and can change my...
6     token from your next the word pawnshop in lieu...
7     a run up in cryptocurrency prices last decembe...
8     the dodo is an extinct flightless bird that wa...
9     dolphins are small toothed cetaceans easily re...
10    elephants are the largest land mammals on eart...
11    bank fixed deposits and government small savin...
12    griego lists you are a green financial profess...
13    jeffrey howard archer baron archer of weston s...
14    the mammoth is a member of an extinct group of...
15    haruki murakami born january twelfth nineteen ...
16    my name is metro for safe microphones are mark...
17    paulo coelho born august twenty fourth nin

In [ ]:
all_transcripts[0][:1100]


"astronomy literally meaning the science that studies the laws of the stars is a natural science that studies celestial objects and phenomena it uses mathematics physics and chemistry in order to explain their origin and evolution objects of interest include planets moons stars nebulae galaxies and comments relevant phenomena include supernova explosions gamma ray bursts quays ours blazers pulsars and cosmic microwave background radiation more generally astronomy studies everything that originates beyond earth's atmosphere cosmology as a branch of astronomy that studies the universe as a whole during the renaissance nicolaus copernicus proposed a heliocentric model of the solar system his work was defended by galileo galilei and expanded upon by johannes kepler kepler was the first to devise a system that correctly described the details of the motion of the plan minutes surround the sun however kepler did not succeed in formulating a theory behind the laws he wrote down one of the olde

#Preprocessing

In [ ]:
text=df['transcript'].values.tolist()
print(len(text))
text

25


["astronomy literally meaning the science that studies the laws of the stars is a natural science that studies celestial objects and phenomena it uses mathematics physics and chemistry in order to explain their origin and evolution objects of interest include planets moons stars nebulae galaxies and comments relevant phenomena include supernova explosions gamma ray bursts quays ours blazers pulsars and cosmic microwave background radiation more generally astronomy studies everything that originates beyond earth's atmosphere cosmology as a branch of astronomy that studies the universe as a whole during the renaissance nicolaus copernicus proposed a heliocentric model of the solar system his work was defended by galileo galilei and expanded upon by johannes kepler kepler was the first to devise a system that correctly described the details of the motion of the plan minutes surround the sun however kepler did not succeed in formulating a theory behind the laws he wrote down one of the old

## 3.1 Lowercasing and Removing Punctuation, Numbers, Special Characters

In [ ]:
#Removing punctuation, numbers special characters and converting to lowercase

stopword=nltk.corpus.stopwords.words('english')
stopword.remove('not')
for index,text_ in enumerate(text):
    text_=re.sub(r'http/S+','',text_) #Removing urls from text 
    text_=re.sub(r'[^A-Za-z#]',' ',text_) #Removing Punctuations, Numbers, and Special Characters
    text_=" ".join(i.lower() for i in text_.split() if i.lower() not in stopword) #Removing stopword
    text[index]=text_
text

['astronomy literally meaning science studies laws stars natural science studies celestial objects phenomena uses mathematics physics chemistry order explain origin evolution objects interest include planets moons stars nebulae galaxies comments relevant phenomena include supernova explosions gamma ray bursts quays blazers pulsars cosmic microwave background radiation generally astronomy studies everything originates beyond earth atmosphere cosmology branch astronomy studies universe whole renaissance nicolaus copernicus proposed heliocentric model solar system work defended galileo galilei expanded upon johannes kepler kepler first devise system correctly described details motion plan minutes surround sun however kepler not succeed formulating theory behind laws wrote one oldest fields astronomy science measurement positions celestial objects historically accurate knowledge positions sun moon planets stars essential celestial navigation use celestial objects guide navigation making ca

## 3.2 Multiple Word removal

In [ ]:
def multiple_word_remove_func(text, words_2_remove_list):
    words_to_remove_list = words_2_remove_list
    words = word_tokenize(text)
    text = ' '.join([word for word in words if word not in words_to_remove_list])
    return text

for index,text_ in enumerate(text):
    text_=multiple_word_remove_func(text_,['a','an','the','they','is','xxx'])
    text[index]=text_
text

['astronomy literally meaning science studies laws stars natural science studies celestial objects phenomena uses mathematics physics chemistry order explain origin evolution objects interest include planets moons stars nebulae galaxies comments relevant phenomena include supernova explosions gamma ray bursts quays blazers pulsars cosmic microwave background radiation generally astronomy studies everything originates beyond earth atmosphere cosmology branch astronomy studies universe whole renaissance nicolaus copernicus proposed heliocentric model solar system work defended galileo galilei expanded upon johannes kepler kepler first devise system correctly described details motion plan minutes surround sun however kepler not succeed formulating theory behind laws wrote one oldest fields astronomy science measurement positions celestial objects historically accurate knowledge positions sun moon planets stars essential celestial navigation use celestial objects guide navigation making ca

## 3.3 Remove Extra Whitespace (More than 1)

In [ ]:
def remove_extra_whitespaces_func(text):
    return re.sub(r'^\s*|\s\s*', ' ', text).strip()

for index,text_ in enumerate(text):
    text_=remove_extra_whitespaces_func(text_)
    text[index]=text_
text

['astronomy literally meaning science studies laws stars natural science studies celestial objects phenomena uses mathematics physics chemistry order explain origin evolution objects interest include planets moons stars nebulae galaxies comments relevant phenomena include supernova explosions gamma ray bursts quays blazers pulsars cosmic microwave background radiation generally astronomy studies everything originates beyond earth atmosphere cosmology branch astronomy studies universe whole renaissance nicolaus copernicus proposed heliocentric model solar system work defended galileo galilei expanded upon johannes kepler kepler first devise system correctly described details motion plan minutes surround sun however kepler not succeed formulating theory behind laws wrote one oldest fields astronomy science measurement positions celestial objects historically accurate knowledge positions sun moon planets stars essential celestial navigation use celestial objects guide navigation making ca

## 3.4 Correcting Misspelt Words

In [ ]:
def spelling_correction(text):
    # Check for spellings in English language
    spell = Speller(lang='en')
    Corrected_text = spell(text)
    return Corrected_text
    
for index,text_ in enumerate(text):
    text_=spelling_correction(text_)
    text[index]=text_
text

['astronomy literally meaning science studies laws stars natural science studies celestial objects phenomena uses mathematics physics chemistry order explain origin evolution objects interest include planets moons stars nebula galaxies comments relevant phenomena include supernova explosions gamma ray bursts quay blazers pillars cosmic microwave background radiation generally astronomy studies everything originates beyond earth atmosphere cosmology branch astronomy studies universe whole renaissance nicolas copernicus proposed heliocentric model solar system work defended galileo galileo expanded upon johannes helper helper first devise system correctly described details motion plan minutes surround sun however helper not succeed formulating theory behind laws wrote one oldest fields astronomy science measurement positions celestial objects historically accurate knowledge positions sun moon planets stars essential celestial navigation use celestial objects guide navigation making calen

## 3.5 Updating back to DF

In [ ]:
dataset['Preprocessed']=text
dataset

,title,transcript,Preprocessed
0,astronomy,astronomy literally meaning the science that s...,astronomy literally meaning science studies la...
1,white dwarf,a white dwarf also called a degenerate dwarf i...,white dwarf also called degenerate dwarf stell...
2,chimp,chimpanzees are great apes found across centra...,chimpanzees great apes found across central we...
3,crypto,what are the top four crypto exchanges india w...,top four crypto exchanges india buy two poker ...
4,crypto_tax,india recently introduced crypto tax in the bu...,india recently introduced crypto tax budget tw...
5,crypto_means,crypto is just one big ponzi and can change my...,crypto one big pond change mind doubt sell wis...
6,digital_token,token from your next the word pawnshop in lieu...,token next word pawnshop lieu small change nex...
7,cryptocurrency,a run up in cryptocurrency prices last decembe...,run cryptocurrency prices last december also d...
8,dodo,the dodo is an extinct flightless bird that wa...,todo extinct flightless bird endemic island vi...
9,dolphin,dolphins are small toothed cetaceans easily re...,dolphins small toothed cetaceans easily recogn...


## 3.6 Handling contractions

In [ ]:
contractions = { 
"ain't": "am not", "aren't": "are not", "can't": "cannot", "can't've": "cannot have", "'cause": "because", "could've": "could have", "couldn't": "could not",
"couldn't've": "could not have", "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hadn't've": "had not have",
"hasn't": "has not", "haven't": "have not", "he'd": "he had", "he'd've": "he would have", "he'll": "he will", "he'll've": "he will have", "he's": "he is",
"how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is", "i'd": "i would", "i'd've": "i would have",
"i'll": "i will", "i'll've": "i will have", "i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",
"it'd've": "it would have", "it'll": "it will", "it'll've": "it will have", "it's": "it is", "let's": "let us",
"ma'am": "madam", "mayn't": "may not", "might've": "might have", "mightn't": "might not", "mightn't've": "might not have", "must've": "must have", "mustn't": "must not",
"mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have", "o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have",
"shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she shall / she will",
"she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have",
"so've": "so have", "so's": "so is", "that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",
"there'd've": "there would have", "there's": "there is", "they'd": "they would", "they'd've": "they would have", "they'll": "they will",
"they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would",
"we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will",
"what'll've": "what will have", "what're": "what are", "what's": "what is", "what've": "what have", "when's": "when is",
"when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have",
"who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",
"would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would", "y'all'd've": "you all would have",
"y'all're": "you all are", "y'all've": "you all have", "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",
"you're": "you are", "you've": "you have"
}

In [ ]:
contractionsWithAnotherInvertedComma = { 
"ain’t": "am not", "aren’t": "are not", "can’t": "cannot", "can’t’ve": "cannot have", "’cause": "because", "could’ve": "could have", "couldn’t": "could not",
"couldn’t’ve": "could not have", "didn’t": "did not", "doesn’t": "does not", "don’t": "do not", "hadn’t": "had not", "hadn’t’ve": "had not have",
"hasn’t": "has not", "haven’t": "have not", "he’d": "he had", "he’d’ve": "he would have", "he’ll": "he will", "he’ll’ve": "he will have", "he’s": "he is",
"how’d": "how did", "how’d’y": "how do you", "how’ll": "how will", "how’s": "how is", "i’d": "i would", "i’d’ve": "i would have",
"i’ll": "i will", "i’ll’ve": "i will have", "i’m": "i am", "i’ve": "i have", "isn’t": "is not", "it’d": "it would",
"it’d’ve": "it would have", "it’ll": "it will", "it’ll’ve": "it will have", "it’s": "it is", "let’s": "let us",
"ma’am": "madam", "mayn’t": "may not", "might’ve": "might have", "mightn’t": "might not", "mightn’t’ve": "might not have", "must’ve": "must have", "mustn’t": "must not",
"mustn’t’ve": "must not have", "needn’t": "need not", "needn’t’ve": "need not have", "o’clock": "of the clock", "oughtn’t": "ought not", "oughtn’t’ve": "ought not have",
"shan’t": "shall not", "shan’t’ve": "shall not have", "she’d": "she would", "she’d’ve": "she would have", "she’ll": "she will",
"she’ll’ve": "she will have", "she’s": "she is", "should’ve": "should have", "shouldn’t": "should not", "shouldn’t’ve": "should not have",
"so’ve": "so have", "so’s": "so is", "that’d": "that would", "that’d’ve": "that would have", "that’s": "that is", "there’d": "there would",
"there’d’ve": "there would have", "there’s": "there is", "they’d": "they would", "they’d’ve": "they would have", "they’ll": "they will",
"they’ll’ve": "they will have", "they’re": "they are", "they’ve": "they have", "to’ve": "to have", "wasn’t": "was not", "we’d": "we would",
"we’d’ve": "we would have", "we’ll": "we will", "we’ll’ve": "we will have", "we’re": "we are", "we’ve": "we have", "weren’t": "were not", "what’ll": "what will",
"what'll've": "what will have", "what're": "what are", "what's": "what is", "what've": "what have", "when's": "when is",
"when’ve": "when have", "where’d": "where did", "where’s": "where is", "where’ve": "where have", "who’ll": "who will", "who’ll’ve": "who will have",
"who’s": "who is", "who’ve": "who have", "why’s": "why is", "why’ve": "why have", "will’ve": "will have", "won’t": "will not", "won’t’ve": "will not have",
"would’ve": "would have", "wouldn’t": "would not", "wouldn’t’ve": "would not have", "y’all": "you all", "y’all’d": "you all would", "y’all’d’ve": "you all would have",
"y’all’re": "you all are", "y’all’ve": "you all have", "you’d": "you would", "you’d’ve": "you would have", "you’ll": "you will", "you’ll’ve": "you will have",
"you’re": "you are", "you’ve": "you have"
}

In [ ]:
class contract():
  def __init__(self, df, contractions, otherContractions):
        self.df = df
        self.contractions = contractions
        self.otherContractions = otherContractions

  def expand(self, text):
        for word in text.split():
            if word in self.contractions.keys():
                text = text.replace(word, self.contractions[word])
            elif word in self.otherContractions.keys():
                text = text.replace(word, self.otherContractions[word])
        return text

  def abbreviate(self, text):
        text = text.split(' ')
        j = 0
        for str_ in text:
            fileName = '/content/Transcripts.csv'
            accessMode = 'r'
            with open(fileName, accessMode) as csvfile:
                dataFromFile = csv.reader(csvfile, delimiter = '=')
                str_ = re.sub('[^a-zA-Z0-9-_.]', '', str_)
                for row in dataFromFile:
                    if str_.upper() == row[0]:
                        text[j] = row[1]
                csvfile.close()
            j += 1
        return ' '.join(text)

  def get(self, text):
        text = self.expand(text)
        text = self.abbreviate(text)
        return text

In [ ]:
pp_class = contract(dataset, contractions, contractionsWithAnotherInvertedComma)
dataset['Preprocessed'] = dataset['Preprocessed']
dataset['Preprocessed'].apply(lambda x : pp_class.get(x))

0     astronomy literally meaning science studies la...
1     white dwarf also called degenerate dwarf stell...
2     chimpanzees great apes found across central we...
3     top four crypto exchanges india buy two poker ...
4     india recently introduced crypto tax budget tw...
5     crypto one big pond change mind doubt sell wis...
6     token next word pawnshop lieu small change nex...
7     run cryptocurrency prices last december also d...
8     todo extinct flightless bird endemic island vi...
9     dolphins small toothed cetaceans easily recogn...
10    elephants largest land mammals earth distinctl...
11    bank fixed deposits government small savings s...
12    grief lists green financial professional compa...
13    jeffrey howard archer baron archer weston supe...
14    mammoth member extinct group elephants mammoth...
15    hari murakami born january twelfth nineteen fo...
16    name metro safe microphones markedly instrumen...
17    paulo cello born august twenty fourth nine

## 3.7 Lemmatization


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Lemma the word
nlp = spacy.load('en', disable=['parser', 'ner'])
def space(comment):
    doc = nlp(comment)
    return " ".join([token.lemma_ for token in doc])

dataset['Preprocessed']= dataset['Preprocessed'].apply(space)
dataset['Preprocessed'].apply(lambda x: x.replace('-PRON-', ' '))

0     astronomy literally mean science study law sta...
1     white dwarf also call degenerate dwarf stellar...
2     chimpanzee great ape find across central west ...
3     top four crypto exchange india buy two poker c...
4     india recently introduce crypto tax budget two...
5     crypto one big pond change mind doubt sell wis...
6     token next word pawnshop lieu small change nex...
7     run cryptocurrency price last december also di...
8     todo extinct flightless bird endemic island vi...
9     dolphin small toothed cetacean easily recogniz...
10    elephant large land mammal earth distinctly ma...
11    bank fix deposit government small saving schem...
12    grief list green financial professional compat...
13    jeffrey howard archer baron archer weston supe...
14    mammoth member extinct group elephant mammoth ...
15    hari murakami bear january twelfth nineteen fo...
16    name metro safe microphones markedly instrumen...
17    paulo cello bear august twenty fourth nine

In [ ]:
dataset.head(8)

,title,transcript,Preprocessed
0,astronomy,astronomy literally meaning the science that s...,astronomy literally mean science study law sta...
1,white dwarf,a white dwarf also called a degenerate dwarf i...,white dwarf also call degenerate dwarf stellar...
2,chimp,chimpanzees are great apes found across centra...,chimpanzee great ape find across central west ...
3,crypto,what are the top four crypto exchanges india w...,top four crypto exchange india buy two poker c...
4,crypto_tax,india recently introduced crypto tax in the bu...,india recently introduce crypto tax budget two...
5,crypto_means,crypto is just one big ponzi and can change my...,crypto one big pond change mind doubt sell wis...
6,digital_token,token from your next the word pawnshop in lieu...,token next word pawnshop lieu small change nex...
7,cryptocurrency,a run up in cryptocurrency prices last decembe...,run cryptocurrency price last december also di...


In [ ]:
dataset.to_csv("preprocessed_dataset.csv", index=False)